In [1]:
import asyncio
from dataclasses import dataclass

from autogen_core import (
    ClosureAgent,
    ClosureContext,
    DefaultSubscription,
    DefaultTopicId,
    MessageContext,
    SingleThreadedAgentRuntime,
)

In [2]:
@dataclass
class FinalResult:
    value: str

In [3]:
queue = asyncio.Queue[FinalResult]()

Create a function closure for outputting the final result to the queue. The function must follow the signature Callable[[AgentRuntime, AgentId, T, MessageContext], Awaitable[Any]] where T is the type of the message the agent will receive. You can use union types to handle multiple message types.

In [4]:
async def output_result(_agent: ClosureContext, message: FinalResult, ctx: MessageContext) -> None:
    await queue.put(message)

In [5]:
runtime = SingleThreadedAgentRuntime()
await ClosureAgent.register_closure(
    runtime, "output_result", output_result, subscriptions=lambda: [DefaultSubscription()]
)

AgentType(type='output_result')

In [6]:
runtime.start()
await runtime.publish_message(FinalResult("Result 1"), DefaultTopicId())
await runtime.publish_message(FinalResult("Result 2"), DefaultTopicId())
await runtime.stop_when_idle()

In [7]:
while not queue.empty():
    print((result := await queue.get()).value)

Result 1
Result 2
